In [ ]:
#import packages
import time
from copy import deepcopy
import json
import pymongo
import os
import datetime
import pandas
import math

In [ ]:
#get the username and and password information
from getpass import getpass

USER = getpass('Enter User Name here: ')
PASS = getpass('Enter Password here: ')
HOST = getpass('Enter HOST here: ')
PORT = getpass('Enter PORT here: ')

Enter User Name here: ··········
Enter Password here: ··········
Enter HOST here: ··········
Enter PORT here: ··········


In [ ]:
#set up mongo connection
mongo_url = "mongodb://{}:{}@{}:{}/?authSource={}".format(USER,
                                                         PASS,
                                                         HOST,
                                                         PORT,
                                                         "lahman")

client = pymongo.MongoClient(mongo_url)

In [ ]:
# Testing connection
db = client['lahman']
#now print all collection names
print("collection names: {}".format(db.list_collection_names()))
reading = db['Batting'].find_one()
#print one batting documents
reading["_id"] = str(reading["_id"])
print(reading)

collection names: ['Salaries', 'AwardsManagers', 'SeriesPost', 'FieldingPost', 'AwardsPlayers', 'HallOfFame', 'SchoolsPlayers', 'Master', 'AwardsShareManagers', 'Fielding', 'Appearances', 'Teams', 'TeamsFranchises', 'Managers', 'FieldingOF', 'Batting', 'Pitching', 'AllstarFull', 'PitchingPost', 'TeamsHalf', 'ManagersHalf', 'Schools', 'AwardsSharePlayers']
{'_id': '6402a053bcf799763cc1028f', 'playerID': 'aardsda01', 'yearID': 2004, 'stint': 1, 'teamID': 'SFN', 'lgID': 'NL', 'G': 11, 'G_batting': 11, 'AB': 0, 'R': 0, 'H': 0, '2B': 0, '3B': 0, 'HR': 0, 'RBI': 0, 'SB': 0, 'CS': 0, 'BB': 0, 'SO': 0, 'IBB': 0, 'HBP': 0, 'SH': 0, 'SF': 0, 'GIDP': 0, 'G_old': 11}


## Queries

1. The number of all stars in allstarfull.
2. The most home runs in a season by a single player (using the batting table).
3. The playerid of the player with the most home runs in a season.
4. The number of leagues in the batting table.
5. Barry Bond's average batting average (playerid = 'bondsba01') where batting average is hits / at-bats. Note you will nead to cast hits to get a decimal: cast(h as real)
6. The teamid with the fewest hits in the year 2000 (ie., yearid = '2000'). Return both the teamid, and the number of hits. Note you can use ORDER BY column and LIMIT 1.
7. The teamid in the year 2000 (i.e., yearid = '2000') with the highest average batting average. Return the teamid and the average. To prevent divsion by 0, limit at-bats > 0.
8. The number of all stars the giants (teamid = 'SFN') had in 2000.
9. The yearid which the giants had the most all stars.
10. The average salary in year 2000.
11. The number of positions (e.g., catchers, pitchers) that have average salaries greather than 2000000 in yearid 2000. You will need to join fielding with salaries.
12. The number of errors Barry Bonds had in 2000.
13. The average salary of all stars in 2000.
14. The average salary of non-all stars in 2000.

In [ ]:
def q1():
  # return the number of all documents in the collection
  return db.AllstarFull.estimated_document_count({}) 

def q2():
  # create a pipeline for aggregation
  pipeline = [
    # group by playerid and find each player's max home run
    {
        "$group": {
            '_id': '$_id',
            'maxhr' : {'$max': '$HR'}
        }
    },
    # ensure no empty value returned
    {
        '$match':{
            'maxhr':{
                '$ne': ''
            }
        }     
    },
    # sort by max home run counts in the collection
    {
        "$sort":{
            "maxhr": pymongo.DESCENDING
        }
    },
    # output the highest number
    {
        '$limit':1
    }
]

  results = db.Batting.aggregate(pipeline)
  # return max hr number
  for r in results:
    return(r['maxhr'])

def q3():
  pipeline = [
    # group by playerid and find each player's max home run
    {
        "$group": {
            '_id': '$_id',
            'playerID': {'$first':'$playerID'},
            'maxhr' : {'$max': '$HR'}
        }
    },
    # ensure no empty value returned
    {
        '$match':{
            'maxhr':{
                '$ne': ''
            }
        }     
    },
    # sort by max home run counts in the collection
    {
        "$sort":{
            "maxhr": pymongo.DESCENDING
        }
    },
    # output the highest number
    {
        '$limit':1
    }
]

  results = db.Batting.aggregate(pipeline)
  # return max hr number player
  for r in results:
    return(r['playerID'])


def q4():
  # use distinct function return all distinct league ID and output the length
  return len(db.Batting.distinct('lgID'))


def q5():
  pipeline = [
    # match the playerid we are interested
    {
        '$match':{
            'playerID': 'bondsba01'
        }     
    },
    # calculate average bat by dividng H to AB
    {
        "$group": {
            '_id': 'null',
            'avgbat' : {'$avg': { '$divide': ['$H' , '$AB']}
                        }
        }
    },
]

  result = db.Batting.aggregate(pipeline)
  for i in result:
    return i['avgbat']


def q6():
  pipeline = [
    # match the year
    {
        '$match':{
            'yearID': 2000
        }     
    },
    # group by teamid and find each team's sum hits
    {
        "$group": {
            '_id': '$teamID',
            'totalhtis': {
                '$sum': '$H'
            }
        }
    },
    # sort by totalhits in ascending order
    {
        "$sort":{
            "totalhtis": pymongo.ASCENDING
        }
    },
    # output the lowest team
    {
        '$limit':1
    }
]

  result = db.Batting.aggregate(pipeline)
  for i in result:
    list_i = [i for i in i.values()]
    return(list_i)


def q7():
  pipeline = [
      # match yearid and ensure AB is greater than 0
    {
        '$match':{
            'yearID': 2000,
            'AB': {'$gt': 0}
            
        }     
    },
    # group by teamid and find each team's average batting
    {
        "$group": {
            '_id': '$teamID',
            'avgbat' : {'$avg': { '$divide': ['$H' , '$AB']}
                        }
            }
    },
     # sort by totalhits in ascendescendingding order
    {
        "$sort":{
            "avgbat": pymongo.DESCENDING
        }
    },
    # output the highest team
    {
        '$limit':1
    }
]

  result = db.Batting.aggregate(pipeline)
  for i in result:
    list_i = [i for i in i.values()]
    return(list_i)

   

def q8():
  pipeline = [
      # match team and year id
    {
        '$match':{
            'yearID': 2000,
            'teamID': 'SFN'
            
        }     
    },
    # group by teamid and find its counts
    {
        "$group": {
            '_id': '$teamID',
            'count': {'$sum': 1}
            }
    }
]

  result = db.AllstarFull.aggregate(pipeline)
  for i in result:
    return(i['count'])



def q9():
  pipeline = [
      # match team id
      {
          '$match':{
              'teamID': 'SFN'
          } 
      },
      # group by year and find distinct stars
      {
          '$group': {
              '_id': "$yearID", 'players': {
                  '$addToSet': '$playerID'
                  }
                  }
       },
      #  caculate the size of dinstic stars
      {
          '$project': {
              '_id': 1, 'yearID':1, 'count': {
                  '$size': "$players"
                  }
                  }
       },
      #  sort by stars from large to small
      {
          '$sort': {
              'count': -1
              }
       },
      #  output the highest number
      {
          '$limit': 1
       }
  ]

  result = db.AllstarFull.aggregate(pipeline)
  for i in result:
    list_i = [i for i in i.values()]
    return(list_i[0])
  

def q10():
  pipeline = [
      # match year id
      {
          '$match':{
              'yearID': 2000
          } 
      },
      # group by year and find average salary
      {
          '$group': {
              '_id': '$yearID', 
              'avgSalary': {
                  '$avg': '$salary'
                  }
                  }
       },
  ]

  result = db.Salaries.aggregate(pipeline)
  for i in result:
    return(i['avgSalary'])



def q11():
  pipeline = [
      # match year id
    {
        '$match':{
            'yearID': 2000
        }
    },
    # inner join with salaries collection to find salary
    {
        '$lookup': {
              'from': 'Salaries',
              'let': 
              {
                'playerID': '$playerID',
                'yearID': '$yearID'
                  },
              'pipeline': [
                  {'$match':
                  {
                      '$expr':
                   {
                       '$and':[
                           {
                              '$eq': ['$playerID', '$$playerID'] 
                           },
                           {
                               '$eq': ['$yearID', '$$yearID']
                           },
                       ]
                   }
                  }
                  },
                  {
                      '$match': {
                          'yearID': 2000
                      }
                  },
                  {
                      '$unset': [
                          'teamID',  'lgID'
                      ]
                  }

              ],
              'as': 'salaries'
          }
    },
    {
        '$unwind': "$salaries"
    },
    # group by position to calculate average salary for each position
    {
        '$group': {
              '_id': '$POS',
              'avgSalary': {
                  '$avg': '$salaries.salary'
              }
          }
    },
    # find those with higher than 2000000 avg salary
    {
    '$match': { 'avgSalary': { '$gt': 2000000 } }
  },
  # output counts
  {
    '$count': "numPositions"
  }
]
  
  result = db.Fielding.aggregate(pipeline)
  for i in result:
    list_i = [i for i in i.values()]
    return(list_i[0])
 


def q12():
  pipeline = [
    {
        # match playerid and yearid
        '$match':{
            'playerID': 'bondsba01',
            'yearID': 2000
        }
    },
    # group by id and calculate total error
    {
        '$group': {
            '_id': 'null',
            'totalE': {
                '$sum': '$E'
            }
        }
    }
]
  
  result = db.Fielding.aggregate(pipeline)
  for i in result:
    return(i['totalE'])



def q13():
  pipeline = [
    {
        # match year
        '$match':{
            'yearID': 2000
        }
    },
    {
        # inner join with salary collection
        '$lookup': {
            'from': 'Salaries',
            'let': 
              {
                'playerID': '$playerID',
                'yearID': '$yearID'
                  },
              'pipeline': [
                  {'$match':
                  {
                      '$expr':
                   {
                       '$and':[
                           {
                              '$eq': ['$playerID', '$$playerID'] 
                           },
                           {
                               '$eq': ['$yearID', '$$yearID']
                           },
                       ]
                   }
                  }
                  },
                  {
                      '$match': {
                          'yearID': 2000
                      }
                  },
                  {
                      '$unset': [
                          'teamID',  'lgID'
                      ]
                  }
              ],
              'as': 'salaries'
          }
        },
        {
        '$unwind': "$salaries"
    },
    # group by year to calculate average salary
    {
        '$group': {
              '_id': '$yearID',
              'avgSalary': {
                  '$avg': '$salaries.salary'
              }
          }
    },
]

  result = db.AllstarFull.aggregate(pipeline)
  for i in result:
    return(i['avgSalary'])
 

def q14():
# find allstar players
  allstars = db.AllstarFull.find({'yearID': 2000 }).distinct("playerID")

  pipeline = [
    {
        # exclude all star players, match year
        '$match':{
            'playerID': {
                '$nin': allstars
            },
            'yearID': 2000
        }
    },
    # calculate average salary of all players left
    {
        '$group': {
            '_id': 'null',
            'avgSalary': {
                '$avg': '$salary'
            }
        }
    }
]

  result = db.Salaries.aggregate(pipeline)
  for i in result:
    return(i['avgSalary'])


In [ ]:
def test_q1():
    result_q1 = q1()
    assert result_q1 == 4834

def test_q2():
    result_q2 = q2()
    assert result_q2 == 73

def test_q3():
    result_q3 = q3()
    assert result_q3 == 'bondsba01'

def test_q4():
    result_q4 = q4()
    assert result_q4 == 7

def test_q5():
    result_q5 = q5()
    assert math.isclose(result_q5, 0.29830576286005, rel_tol=1e-6)

def test_q6():
    result_q6 = q6()
    assert result_q6[0] == 'MIL'
    assert result_q6[1] == 1366

def test_q7():
    result_q7 = q7()
    assert result_q7[0] == 'SEA'
    assert math.isclose(result_q7[1], 0.280799158489005, rel_tol=1e-6)

def test_q8():
    result_q8 = q8()
    assert result_q8 == 2

def test_q9():
    result_q9 = q9()
    assert result_q9 in [1961, 1962, 1966]

def test_q10():
    result_q10 = q10()
    assert int(result_q10) == 1992984

def test_q11():
    result_q11 = q11()
    assert result_q11 == 3

def test_q12():
    result_q12 = q12()
    assert result_q12 == 6

def test_q13():
    result_q13 = q13()
    assert int(result_q13) == 5388841

def test_q14():
    result_q14 = q14()
    assert int(result_q14) == 1692309

In [ ]:
test_q1()
print('Q1 - succesful')
test_q2()
print('Q2 - succesful')
test_q3()
print('Q3 - succesful')
test_q4()
print('Q4 - succesful')
test_q5()
print('Q5 - succesful')
test_q6()
print('Q6 - succesful')
test_q7()
print('Q7 - succesful')
test_q8()
print('Q8 - succesful')
test_q9()
print('Q9 - succesful')
test_q10()
print('Q10 - succesful')
test_q11()
print('Q11 - succesful')
test_q12()
print('Q12 - succesful')
test_q13()
print('Q13 - succesful')
test_q14()
print('Q14 - succesful')

print('All asserts are successful!')

Q1 - succesful
Q2 - succesful
Q3 - succesful
Q4 - succesful
Q5 - succesful
Q6 - succesful
Q7 - succesful
Q8 - succesful
Q9 - succesful
Q10 - succesful
Q11 - succesful
Q12 - succesful
Q13 - succesful
Q14 - succesful
All asserts are successful!
